In [1]:
N_RSA_BITS = 1024
ASN1_SHA1 = b'\x30\x21\x30\x09\x06\x05\x2b\x0e\x03\x02\x1a\x05\x00\x04\x14'

from hashlib import sha1
from Crypto.Util.number import getPrime

Prime = lambda : getPrime(N_RSA_BITS//2)

class RSASigner(object):
    def __init__(self, e = 3):
        self.e = e
        p, q, et = GeneratePrimes(e)
        self.p = p
        self.q = q
        self.d = ModInv(e, et)
        self.n = p*q
        
    def GetPubkey(self):
        return self.e, self.n
    
    def DecryptBytes(self, c):
        m = pow(c, self.d, self.n)
        return m.to_bytes(byteorder='big', length = N_RSA_BITS//8)
    
    def DecryptInt(self, c):
        m = pow(c, self.d, self.n)
        return m
    
    def Sign(self, message: bytes):
        digest = sha1(message).digest()
        npad = N_RSA_BITS//8 - 2 - len(ASN1_SHA1) - len(digest)
        x = b'\x00\x01' + b'\xFF'*npad + ASN1_SHA1 + digest
        assert len(x) == N_RSA_BITS//8
        return self.DecryptInt( int.from_bytes(x, byteorder='big') )
        

class RSAVerifier(object):
    def __init__(self, e, n):
        self.e = e
        self.n = n
        
    def Encrypt(self, message):
        m = message if type(message) is int else int.from_bytes(message, byteorder = 'big')
        assert m < self.n
        return pow(m, self.e, self.n)
    
    def Verify(self, message: bytes, signature: int):
        digest = sha1(message).digest()
        x = pow(signature, self.e, self.n)
        assert x.to_bytes(byteorder='big', length = N_RSA_BITS//8).endswith(digest)
    
# Using implementations from https://github.com/ricpacca/cryptopals/blob/master/S5C39.py

def GCD(a, b):
    """Computes the greatest common divisor between a and b using the Euclidean algorithm."""
    while b != 0:
        a, b = b, a % b

    return a


def LCM(a, b):
    """Computes the lowest common multiple between a and b using the GCD method."""
    return a // GCD(a, b) * b


def ModInv(a, n):
    """Computes the multiplicative inverse of a modulo n using the extended Euclidean algorithm."""
    t, r = 0, n
    new_t, new_r = 1, a

    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        raise Exception("a is not invertible")
    if t < 0:
        t = t + n

    return t

def GeneratePrimes(e = 3):
    while True:
        p, q = Prime(), Prime()
        n = p*q
        et = LCM(p-1, q-1) % n

        if 2 < e < et and GCD(e, et) == 1:
            return p, q, et

assert ModInv(17, 3120) == 2753

In [2]:
signer = RSASigner()
pubkey = signer.GetPubkey()
verifier = RSAVerifier(*pubkey)

In [3]:
message = b'hi mom'

signature = signer.Sign(message)
print(signature)

72372505392292995953870657782337420035744586597900550646974001036808223151679809974646438084912360454561080783027119302999911397589496984765742129096992320745123818726361648563754898404995445380421624995367369003010842704842466640363340171862270111444722366788801706279005776937827751702971986770256506428900


In [4]:
verifier.Verify(message, signature)